# TO DO

- View information on the spam data,
- View the length of messages,
- Define a function to eliminate stopwords,
- Apply Bag of Words,
- Apply tf-idf transformer, and
- Detect Spam with Naïve Bayes model.

In [3]:
import pandas as pd
import sklearn 
import widgetsnbextension

import string
from nltk.corpus import stopwords

In [ ]:
#Get the spam data collection 

In [15]:
df_spam = pd.read_csv('SpamCollection',sep='\t', names =['response', 'message'], header = None)

In [16]:
#view response 
df_spam.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [49]:
df_spam.describe()

,length
count,5572.000000
mean,80.489950
std,59.942907
min,2.000000
25%,36.000000
50%,62.000000
75%,122.000000
max,910.000000


In [50]:
df_spam.groupby('response').describe()

length                                                         
           count        mean        std   min    25%    50%    75%    max
response                                                                 
ham       4825.0   71.482487  58.440652   2.0   33.0   52.0   93.0  910.0
spam       747.0  138.670683  28.873603  13.0  133.0  149.0  157.0  223.0

In [14]:
#Verify length of the messages and also add it as a new column 

In [51]:
df_spam['length'] = df_spam['message'].apply(len)
df_spam.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [54]:
#define a function to get rid of stopwords present in the messages

def remove_stopwords(sentence):
    #This function returns a list with the 'meaningful' WORDS in the sentence
    
    #1. - remove punctuation
    no_punctuation = [char for char in sentence if char not in string.punctuation]
    #2. - join chars
    no_punctuation = ''.join(no_punctuation)
    #3. - split words
    words_sentence = no_punctuation.split()
    #4. - remove stopwords
    clean_words = [word for word in words_sentence if word.lower() not in stopwords.words('english')]
    
    return clean_words

In [55]:
df_spam['message'].head().apply(remove_stopwords)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [58]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer #numerical value to each word in the text
vectorizer = CountVectorizer(analyzer = remove_stopwords) #will use our function as analyzer

In [61]:
#use bag of words by applying the function and fit the data into it
bag_of_words_transformer = vectorizer.fit(df_spam['message']) # vectorize transform - applied to the sentences
print(bag_of_words_transformer) 

CountVectorizer(analyzer=<function remove_stopwords at 0x7f8f549cfaf0>)


In [62]:
#print length of bag of words stored in the vocabulary_ attribute
len(bag_of_words_transformer.vocabulary_)

11425

In [63]:
# store bag of words for messages using the transform method
messsage_bag_of_words = bag_of_words_transformer.transform(df_spam['message'])

In [64]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messsage_bag_of_words)
# tf = term frequency idf = inverse document frequence

In [66]:
#print shape of the tfidf 
tfidf_message = tfidf_transformer.transform(messsage_bag_of_words)
print(tfidf_message.shape)

(5572, 11425)


In [70]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB

spam_detect_model = MultinomialNB().fit(tfidf_message,df_spam['response'])

In [80]:
#check model for the predicted and expected value say for message#2 and message#5

n = 5 #message to check

message_to_predict = df_spam['message'][n]
bag_of_words = bag_of_words_transformer.transform([message_to_predict])
tfidf = tfidf_transformer.transform(bag_of_words)

print('expected', df_spam['response'][n])
print('predicted', spam_detect_model.predict(tfidf[0]))

expected spam
predicted ['ham']
